# KMeans


## Concept

L'algorithme des K-Means fait parti des algorithmes non supervisés de clustering. 
Un algorithme non supervisé cherche des patterns ou des structures dans les données, il ne cherche pas à prédire une variable.

La méthode K-Means vise à répartir N observations en K groupes dans lesquels chaque observation appartient au groupe dont la moyenne est la plus proche (centroïde du groupe).

En d'autres termes, il minimise la dissimilarité à l'intérieur des clusters en utilisant la fonction objective :

>$ \min_{C_1, C_2,..., C_k} \sum^{k}_{i = 1} \sum_{x \in C_i} \left\| x - u_i \right\| ^{2} ~ avec ~ u_i = \frac{1}{\lvert C_i \rvert} \sum_{x \in C_i} x$

Moins formellement: 
>Classes tels quel argminSomme des( Sommes des distances intra-classe)

- C représente chaque groupe, de 1 à K 
- $u_i$ repésente le centroïde du groupe i
- $x \in C_i$ est l'ensemble des points associés au cluster $C_i$

## L'Algorithme

*Début*:
>Choisir K points du nuage de points

À répéter jusqu'à convergence ou stabilisation de l’inertie:
>- Affecter chaque point (élément de la matrice de données) au groupe dont il est le plus proche à son centre.
>- Recalculer le centre de chaque cluster  et modifier le centroïde.


[Preuve de la consistance](http://www.stat.yale.edu/~pollard/Papers/Pollard81AS.pdf)

## Options:
Dans l'algorithme des KMeans, l'initialisation est importante, le nombre de boucles avant convergence en est très dépendantes.

Il existe plusieurs méthodes d'initialisation, en voici deux parmis les plus populaires:
- Méthode aléatoires: On choisit les K centroïdes aléatoirement parmis les points du nuage de points
- Méthode Kmeans++: On choisit des points pour centoïdes 'éloignés les uns des autres'

Il est également possible d'utilisé d'autres distance que la distance euclidienne, tel que la distance de Manhattan.

## En application:

>Dans un objectif pédagogique nous allons utiliser des toysets et non des données réel.

In [ ]:
## Installation des modules si nécessaire
%pip install -U scikit-learn
%pip install plotly

In [ ]:
#Importation des modules
import numpy as np
import pandas as pd
from sklearn.datasets import make_circles,make_moons,make_blobs
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.cluster import KMeans
import plotly.express as px

#### On crée un toyset

In [ ]:
#Création des données pour visualiser

df_blob=pd.DataFrame()
X, y_true = make_blobs(
    n_samples=300, centers=4, cluster_std=0.50, random_state=0
)
df_blob['x'],df_blob['y'],df_blob['color']=X[:,0],X[:,1],y_true
px.scatter(x=df_blob['x'],y=df_blob['y'],color=df_blob['color'])

In [ ]:
#Fonction pour visualiser les espaces d'attribution du kmeans
def plot_decision(clf,df):
    lrange = np.arange(min(df['x']), 1.5*max(df['x']), .02)
    wrange = np.arange(min(df['y']), max(df['y']), .02)
    ll, ww = np.meshgrid(lrange, wrange)
    pred = clf.predict(np.c_[ll.ravel(), ww.ravel()])
    fig=go.Heatmap(
        x=ll.ravel(),
        y=ww.ravel(),
        z=pred,  #<-Sureté d'attribution selon le classifieur
        colorscale='RdBu',#Echelle de couleur utilisée
        opacity=0.5,#Transparence du fond
        customdata=pred,#Pour avoir des données en + 
        hovertemplate='''Abscisse: %{x:.3f}<br>Ordonnée: %{y:.3f}
               <br>Classe: %{customdata:.3f} ''',visible=False
        #affichage sur l'étiquette
    )
    return fig

#### Visualisation de l'évolution des clusters par les KMeans dans le cas où une spéaration linéaire est possible

In [ ]:
import warnings
warnings.filterwarnings('ignore')

fig = go.Figure()

for step in range(1,10):
    kms_c = KMeans(n_clusters=4, init=df_blob[df_blob['color']==0][['x','y']][4:8].values, n_init=1, max_iter=step, tol=0.00001, verbose=0, random_state=None, copy_x=True, algorithm='auto').fit(df_blob[['x','y']])
    fig.add_trace(plot_decision(kms_c,df_blob))
    fig.add_trace(go.Scatter(mode='markers',x=kms_c.cluster_centers_[:,0],y=kms_c.cluster_centers_[:,1],marker_color='black',marker_size=10))

fig.data[0].visible = True
fig.add_trace(go.Scatter(mode='markers',x=df_blob['x'],y=df_blob['y'],marker_color=df_blob['color']))
steps = []
for i in range(int((len(fig.data))/2)):
    step = dict(
        method="update",
        args=[{"visible": [False] * (len(fig.data))},
              {"title": "Cluster après : " + str(i)+' étapes'}],
    )
    step["args"][0]["visible"][2*i] = True
    step["args"][0]["visible"][2*i+1] = True
    step["args"][0]["visible"][len(fig.data)-1] = True
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Nombres de tours de boucles: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.show()


In [ ]:
del fig #Pour regagner un peu de mémoire

### Maintenant plaçons nous dans le cas où une séparation linéaire n'est pas possible:

In [ ]:
A=make_circles(n_samples=1000, shuffle=True, noise=0.003, random_state=None,factor=0.2)
df_circle = pd.DataFrame({'x':A[0][:,0],'y':A[0][:,1],'color':A[1]})


A=make_moons(n_samples=1000, shuffle=True, noise=None, random_state=None)
df_moon = pd.DataFrame({'x':A[0][:,0],'y':A[0][:,1],'color':A[1]})
del A

px.scatter(x=df_circle['x'],y=df_circle['y'],color=df_circle['color'])

### Voyons comment évolue notre algorithme de K-Means sur de tels données

In [ ]:
fig = go.Figure()

for step in range(1,10):
    kms_c = KMeans(n_clusters=2, init=df_circle[df_circle['color']==0][['x','y']][:2].values, n_init=1, max_iter=step, tol=0.00001, verbose=0, random_state=None, copy_x=True, algorithm='auto').fit(df_circle[['x','y']])
    fig.add_trace(plot_decision(kms_c,df_circle))
    fig.add_trace(go.Scatter(mode='markers',x=kms_c.cluster_centers_[:,0],y=kms_c.cluster_centers_[:,1],marker_color='black',marker_size=10))

fig.data[0].visible = True
fig.add_trace(go.Scatter(mode='markers',x=df_circle['x'],y=df_circle['y'],marker_color=df_circle['color']))
steps = []
for i in range(int((len(fig.data))/2)):
    step = dict(
        method="update",
        args=[{"visible": [False] * (len(fig.data))},
              {"title": "Cluster après : " + str(i)+' étapes'}],
    )
    step["args"][0]["visible"][2*i] = True
    step["args"][0]["visible"][2*i+1] = True
    step["args"][0]["visible"][len(fig.data)-1] = True
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Nombres de tours de boucles: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.show()


In [ ]:
del fig

*On voit très clairement que les clusters formées ne sont pas convenables* ( Même si ici, un choix désastreux pour l'initialisation n'arrange pas les choses)

C'est dans ce genre de situations que l'utilisation de la version kernel des kmeans est nécessaire.

## Concept

### L'idée derrière les kernels kmeans est de ne plus considérer les coordonnées de notre nuage de point mais de considérer les coordonnées engendrés par le noyau de nos points, pour être un peu plus clair, on va utiliser quelques exemples assez simple.

#### Par visualisation vis à vis de deux points

In [ ]:
from sklearn.metrics.pairwise import pairwise_kernels
fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.6, 0.4],
    specs=[[{"type": "scatter"}, {"type": "scatter"}]])

p_d_b=pd.DataFrame(np.random.rand(15,2))
p_d_b['indice']=[i for i in range(len(p_d_b))]
n_pdb=p_d_b.copy()
n_pdb[[0,1]]=pairwise_kernels(X=p_d_b[[0,1]],Y=[[1,0],[0,1]],metric='rbf')
fig.add_trace(go.Scatter(mode='markers+text',x=p_d_b[0],y=p_d_b[1],marker_color=p_d_b['indice'],text=p_d_b['indice']),row=1,col=1)
fig.add_trace(go.Scatter(mode='markers+text',x=n_pdb[0],y=n_pdb[1],marker_color=n_pdb['indice'],text=n_pdb['indice']),row=1,col=2)

fig.show()

#### Visualisation de la heatmap permuté du kernel des points de df_circle

In [ ]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import reverse_cuthill_mckee
mat=df_circle[['x','y']].values
def visuel(L,cut):
    def change(B):
        A=B.copy()
        graph = csr_matrix(A)
        perm = reverse_cuthill_mckee(graph,symmetric_mode=True)
        for i in range(len(perm)):
            A[:,i]=A[perm,i]
        for i in range(len(perm)):   
            A[i,:]=A[i,perm]
        return A
    A=L.copy()
    A[A<cut]=0
    Lr=change(A)
    fig = px.imshow(Lr)
    return fig
visuel(pairwise_kernels(X=mat,metric='rbf',gamma=15),0.2).show()

#### Je pense que la matrice ci dessus permets bien de voir la présence de deux clusters, même si, en pratique, la différence n'est pas aussi visible.

Très clairement on voit qu'une moitié des points ont pour coordonnées significatives une moitié des axes et l'autre moitié des points l'autre moitié des axes.

## Application du kernel k-means

In [ ]:
def plot_decision_kernel(clf,df,**kwargs):
    '''
    Fonction pour afficher en fond les cluster attribués par le kernel-kmean
    '''
    lrange = np.arange(min(df['x']), 1.5*max(df['x']), .02)
    wrange = np.arange(min(df['y']), max(df['y']), .02)
    ll, ww = np.meshgrid(lrange, wrange)
    pred = clf.predict(pairwise_kernels(X=np.c_[ll.ravel(), ww.ravel()],Y=df[['x','y']],**kwargs))
    fig=go.Heatmap(
        x=ll.ravel(),
        y=ww.ravel(),
        z=pred,  #<-Cluster prédit
        colorscale='RdBu',#Echelle de couleur utilisée
        opacity=0.5,#Transparence du fond
        customdata=pred,#Pour avoir des données en + 
        hovertemplate='''Abscisse: %{x:.3f}<br>Ordonnée: %{y:.3f}
               <br>Classe: %{customdata:.3f} '''
        #affichage sur l'étiquette
    )
    return fig

fig = make_subplots(
    rows=4, cols=2,
    specs=[[{"type": "scatter"}, {"type": "scatter"}],
          [{"type": "scatter"}, {"type": "scatter"}],
          [{"type": "scatter"}, {"type": "scatter"}],
          [{"type": "scatter"}, {"type": "scatter"}]],
subplot_titles=("Cercle", "Moon","Cercle rbf:1", "Moon rbf:1","Cercle rbf:15", "Moon rbf:15","Cercle rbf:100", "Moon rbf:100"))
fig.add_trace(go.Scatter(mode='markers',x=df_circle['x'],y=df_circle['y'],marker_color=df_circle['color']),row=1,col=1)
fig.add_trace(go.Scatter(mode='markers',x=df_moon['x'],y=df_moon['y'],marker_color=df_moon['color']),row=1,col=2)
for i in zip([1,15,100],[2,3,4]):
    kernel=pairwise_kernels(df_circle[['x','y']],metric='rbf',gamma=i[0])
    clf=KMeans(n_clusters=2, init='k-means++', n_init=10, max_iter=300, tol=0.0001, verbose=0, random_state=None, copy_x=True, algorithm='auto').fit(kernel)
    fig.add_trace(plot_decision_kernel(clf,df_circle,metric='rbf',gamma=i[0]),col=1,row=i[1])
    fig.add_trace(go.Scatter(mode='markers',x=df_circle['x'],y=df_circle['y'],marker_color=clf.labels_),col=1,row=i[1])
    
for i in zip([1,15,100],[2,3,4]):
    kernel=pairwise_kernels(df_moon[['x','y']],metric='rbf',gamma=i[0])
    clf=KMeans(n_clusters=2, init='k-means++', n_init=100, max_iter=30000, tol=0.0001, verbose=0, random_state=0, copy_x=True, algorithm='auto').fit(kernel)
    fig.add_trace(plot_decision_kernel(clf,df_moon,metric='rbf',gamma=i[0]),col=2,row=i[1])
    fig.add_trace(go.Scatter(mode='markers',x=df_moon['x'],y=df_moon['y'],marker_color=clf.labels_),col=2,row=i[1])

In [ ]:
fig.update_layout(template='plotly_dark',showlegend=False)
fig.show(renderer='browser',title_text="Plot des frontière de cluster selon noyau")

## Question: Comment choisir le nombre de cluster ?
[Source](https://medium.com/analytics-vidhya/how-to-determine-the-optimal-k-for-k-means-708505d204eb)

### Méthode du coude sur l'inertie

In [ ]:
#Sur le blob
px.line(x=[i for i in range(1,10)],y=[KMeans(n_clusters=i, init='k-means++', n_init=10).fit(df_blob[['x','y']]).inertia_ for i in range(1,10)])

#### On prends le coude le plus resseré

### Méthode de la silhouette

In [ ]:
from sklearn.metrics import silhouette_score

sil = []
kmax = 10

# dissimilarity would not be defined for a single cluster, thus, minimum number of clusters should be 2
for k in range(2, kmax+1):
    kmeans = KMeans(n_clusters = k).fit(df_blob[['x','y']])
    labels = kmeans.labels_
    sil.append(silhouette_score(df_blob[['x','y']], labels, metric = 'euclidean'))
    
px.line(x=[i for i in range(1,10)],y=sil)

### On prend la valeurs la plus haute

## Question: Comment choisir le noyau ?

### Je n'ai pas trouvé de guide de bonnes pratiques, cependant:
#### /!\ Intuition personnelle, rien de fondé
#### J'ai l'impression qu'avec une visualisation tel que celle de la matrice du kernel, on peut observer quelques petites chose:

In [ ]:
mat=mat=df_moon[['x','y']].values
visuel(pairwise_kernels(X=mat,metric='rbf',gamma=1),0.1).show()

In [ ]:
visuel(pairwise_kernels(X=mat,metric='rbf',gamma=15),0.1).show()

In [ ]:
visuel(pairwise_kernels(X=mat,metric='rbf',gamma=100),0.1).show()